# Data Pipelines with Python Project ~ P.M. Kuria

## 1. Defining the Question

### a) Specifying the Data Analysis Question

As a Data professional, you are required to create an automated data pipeline that can extract billing data from multiple sources and transform it into a structured format for efficient analysis and revenue reporting.

### b) Defining the Metric for Success

Measure of success will be measured by atttempting to have the data pipeline do the following: 


1. Extract the data: Use Python to read the CSV files and extract the data.
2. Clean the data: Perform data cleaning on the extracted data to remove any missing
values and outliers. For example, you can replace missing values with an appropriate
value or remove them altogether.
3. Transform the data: Apply any necessary transformations on the data, such as data
type conversion, data aggregation, and data filtering, to prepare the data for analysis.
4. Merge the datasets: Join the different datasets into a single dataset that can be used for
analysis.
5. Load the data: Load the transformed data into a database or a file, such as a CSV file,
that can be easily analyzed.
6. Automate the process: Automate the data pipeline by scheduling it to run at a specific
time, such as daily or weekly so that it can update the analysis data automatically.
7. Test the pipeline: Test the data pipeline to ensure it produces the correct results. This can be done by comparing the results with the expected output or using a test dataset.
8. Optimize the pipeline: Optimize the data pipeline to improve performance and reduce errors. This can be done by optimizing the code, parallel processing, and reducing the data size.
9. Monitor the pipeline: Monitor the data pipeline to ensure that it runs smoothly and that there are no errors or issues.


### c) Understanding the context 

Telecom companies often have to extract billing data from multiple CSV files generated from various systems and transform it into a structured format for analysis and revenue reporting. This process can be time-consuming, error-prone, and hinder decision-making. Manually analyzing and reconciling billing data from different sources is a tedious task and often leads to delays in generating revenue reports. Thus, there is a need for an automated data pipeline that can extract billing data from multiple sources and transform it into a structured format for efficient analysis and revenue reporting.

Datasets

Here are three sample datasets (https://bit.ly/416WE1X) with billing data that can be joined. The datasets contain some missing values and outliers:
Dataset 1:

● Customer ID (numeric)

● Date of purchase (MM/DD/YYYY)

● Total amount billed (numeric)

● Payment status (categorical - paid, overdue, disputed)

● Payment method (categorical - credit card, bank transfer, e-wallet)
● Promo code (text)

● Country of purchase (categorical)


Dataset 2:

● Customer ID (numeric)

● Date of payment (MM/DD/YYYY)

● Amount paid (numeric)

● Payment method (categorical - credit card, bank transfer, e-wallet)

● Payment status (categorical - paid, overdue, disputed)

● Late payment fee (numeric)

● Country of payment (categorical)


Dataset 3:

● Customer ID (numeric)

● Date of refund (MM/DD/YYYY)

● Refund amount (numeric)

● Reason for refund (text)

● Country of refund (categorical)

Notes:

1. The datasets can be joined using Customer ID, Date of purchase/payment/refund, and country of purchase/payment/refund as keys.
2. The datasets may contain missing values and outliers for some fields, such as the total amount billed or refund amount.
3. The payment status may be missing or incomplete for some of the transactions.
4. The promo code field may be empty for some of the purchases.
5. The reason for the refund may be missing for some of the refund transactions.

### d) Recording the Experimental Design

1. Data Importation.
2. Data Cleaning.
3. Data Tranformation.
4. Run the pipeline.

### e) Data Relevance

The dataset used is indisputable, since it is available publicly online and anyone can use the same data to challenge any conclusion arrived at on the analysis performed herein.

## 2. Data Importation


In [9]:
# install pandas
import pandas as pd
import os

In [45]:
# Define the names of the CSV files
csv_file1 = 'dataset1.csv'
csv_file2 = 'dataset2.csv'
csv_file3 = 'dataset3.csv'

# Load the datasets
dataset1 = pd.read_csv(csv_file1)
dataset2 = pd.read_csv(csv_file2)
dataset3 = pd.read_csv(csv_file3)

In [25]:
dataset1.columns


Index(['customer_id', 'date_of_purchase', 'total_amount_billed',
       'payment_status', 'payment_method', 'promo_code',
       'country_of_purchase'],
      dtype='object')

In [26]:
dataset2.columns


Index(['customer_id', 'date_of_payment', 'amount_paid', 'payment_method',
       'payment_status', 'late_payment_fee', 'country_of_payment'],
      dtype='object')

In [27]:
dataset3.columns


Index(['customer_id', 'date_of_refund', 'refund_amount', 'reason_for_refund',
       'country_of_refund'],
      dtype='object')

## 3. Data Cleaning

In [32]:
# Clean the data
# Replace missing values in Total amount billed with 0
dataset1['total_amount_billed'].fillna(value=0, inplace=True)
# Replace missing values in Refund amount with 0
dataset3['refund_amount'].fillna(value=0, inplace=True)

## Mereging

In [50]:
# Merge the datasets

# Rename the columns
dataset1 = dataset1.rename(columns={"date_of_purchase": "date", "country_of_purchase": "country"})
dataset2 = dataset2.rename(columns={"date_of_payment": "date", "country_of_payment": "country"})
dataset3 = dataset3.rename(columns={"date_of_refund": "date", "country_of_refund": "country"})


merged_dataset = pd.merge(dataset1, dataset2, on=['customer_id','date','country'],
                          how='outer', suffixes = ('_purchase', '_payment'))
merged_dataset = pd.merge(merged_dataset, dataset3, on=['customer_id','date','country'], 
                          how='outer', suffixes = ('_pp', '_refund'))

merged_dataset.head()

,customer_id,date,total_amount_purchase,payment_status_purchase,payment_method_purchase,promo_code,country,total_amount_payment,payment_method_payment,payment_status_payment,late_payment_fee,total_amount,reason_for_refund
0,101,04/01/2021,100.0,paid,credit card,PROMO1,USA,100.0,credit card,paid,0.0,NaN,NaN
1,102,04/02/2021,200.0,paid,bank transfer,PROMO2,USA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,04/02/2021,50.0,overdue,credit card,NaN,UK,NaN,NaN,NaN,NaN,NaN,NaN
3,104,04/03/2021,75.0,disputed,e-wallet,PROMO3,UK,NaN,NaN,NaN,NaN,NaN,NaN
4,105,04/04/2021,125.0,paid,credit card,PROMO4,USA,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
merged_dataset.columns

Index(['customer_id', 'date', 'total_amount_purchase',
       'payment_status_purchase', 'payment_method_purchase', 'promo_code',
       'country', 'total_amount_payment', 'payment_method_payment',
       'payment_status_payment', 'late_payment_fee', 'total_amount',
       'reason_for_refund'],
      dtype='object')

In [52]:
# Clean the data further
# Replace missing values in Amount paid with 0
merged_dataset['total_amount_payment'].fillna(value=0, inplace=True)
# Replace missing values in Late payment fee with 0
merged_dataset['late_payment_fee'].fillna(value=0, inplace=True)
# Replace missing values in Promo code with 'NA'
merged_dataset['promo_code'].fillna(value='NA', inplace=True)
# Replace missing values in Reason for refund with 'NA'
merged_dataset['reason_for_refund'].fillna(value='NA', inplace=True)

## 4. Data transformation

In [55]:
# Transform the data
merged_dataset['date'] = pd.to_datetime(merged_dataset['date'], format='%m/%d/%Y')

# Convert Total amount billed, Amount paid, Refund amount, and Late payment fee to float
merged_dataset['total_amount_purchase'] = pd.to_numeric(merged_dataset['total_amount_purchase'], errors='coerce')
merged_dataset['total_amount_payment'] = pd.to_numeric(merged_dataset['total_amount_payment'], errors='coerce')
merged_dataset['total_amount'] = pd.to_numeric(merged_dataset['total_amount'], errors='coerce')
merged_dataset['late_payment_fee'] = pd.to_numeric(merged_dataset['late_payment_fee'], errors='coerce')


In [56]:
# Aggregate the data
# Compute the difference between Total amount billed and Amount paid as Balance due
merged_dataset['balance_due'] = merged_dataset['total_amount_purchase'] - merged_dataset['total_amount_payment']


In [57]:
# Filter the data
# Keep only the rows where Balance due is greater than 0
merged_dataset = merged_dataset[merged_dataset['balance_due'] > 0]

## 5. Save final pipeline output

In [58]:
# Output the data to a CSV file
merged_dataset.to_csv('billing_data.csv', index=False)

### a). Did we have the right data?

Yeah, to answer the questions given.

### b). Do we need other data to answer our question?

For the required analysis this data is sufficient for now.

### c). Did we have the right question?

Yes as to meet the brief.